In [1]:
import numpy as np
from numba.typed import List as NumbaList
from BPnumba.NumAG import TrainAG
from BPmodule.BPGen  import PROBLEM2,PROBLEM3,PROBLEM4,PROBLEM5
from BPnumba.GeneticOperators import CreatePoblation,InstancePob
from BPnumba.DFFA import DFFtrain


In [4]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]
pop=50
contenedor =np.asanyarray( P3A1[0],dtype=np.int64)
datos =np.asanyarray(P3A1[1],dtype=np.int64)
idBoxes = np.arange(1,len(datos)+1,dtype=np.int64)
initial = CreatePoblation(pop,idBoxes)
gamma= 1/len(datos)
maxItr=500

In [5]:
Pob = InstancePob(NumbaList(initial),datos,contenedor)

In [6]:
best =DFFtrain(2,NumbaList(Pob),gamma,NumbaList(datos),NumbaList(contenedor))

In [8]:
best =DFFtrain(maxItr,NumbaList(Pob),gamma,NumbaList(datos),NumbaList(contenedor))

In [9]:
best.fi

0.994825

In [10]:
best =DFFtrain(maxItr*2,NumbaList(Pob),gamma,NumbaList(datos),NumbaList(contenedor))
best.fi

0.991757